## Applied ML for baseball:

#### By Thomas Maxence Franco 
Submitted to the Faculty of Science in partial fulfillment of the requirements for the degree of 
#### Master of Modeling for Science and Engineering 
at the 
#### UNIVERSITAT AUTÒNOMA DE BARCELONA 
Directed by 
Tomás Manuel Margalef Burrull
July 2024


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from itertools import combinations

In [2]:
file_path = "C:\\Users\\mfran\\OneDrive - UAB\\Masters\\Thesis\\v2\\tables\\tradfinalbat.csv"
df = pd.read_csv(file_path)


In [3]:
df.head()

,Name,contract_years,salary,AAV,minor_league,catcher,date,new_team,former_team,Age,...,SB_2,BB%_2,K%_2,AVG_2,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2
0,Shohei Ohtani,10,700.0,70.000000,0,0,2024,LAD,LAA,29,...,11,0.108108,0.241742,0.273038,0.355856,0.518771,6,9,160,0.874627
1,Aaron Judge,9,360.0,40.000000,0,0,2023,NYY,NYY,31,...,6,0.118483,0.249605,0.287273,0.372828,0.543636,16,1,158,0.916464
2,Carlos Correa,3,105.3,35.100000,0,0,2022,MIN,HOU,27,...,1,0.109034,0.233645,0.278571,0.358255,0.567857,8,0,78,0.926113
3,Anthony Rendon,7,245.0,35.000000,0,0,2020,LAA,WSN,30,...,2,0.092127,0.137353,0.308129,0.373534,0.534972,5,1,163,0.908506
4,Carlos Correa,6,200.0,33.333333,0,0,2023,MIN,MIN,28,...,0,0.117188,0.181250,0.279279,0.365625,0.484685,16,0,155,0.850310


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              677 non-null    object 
 1   contract_years    677 non-null    int64  
 2   salary            677 non-null    float64
 3   AAV               677 non-null    float64
 4   minor_league      677 non-null    int64  
 5   catcher           677 non-null    int64  
 6   date              677 non-null    int64  
 7   new_team          677 non-null    object 
 8   former_team       677 non-null    object 
 9   Age               677 non-null    int64  
 10  WAR3              677 non-null    float64
 11  Yrs               677 non-null    int64  
 12  career_games      677 non-null    int64  
 13  stayed_same_team  677 non-null    int64  
 14  G                 677 non-null    int64  
 15  PA                677 non-null    int64  
 16  HR                677 non-null    int64  
 1

Null values where handled in a previous step manually. The small amount of null values was due to players not playing through a season because of injury, but that would damage both of the season as we need last season's data to fill the second to last. I had to find each player's stats in order to not lose any data. This can be handled differently to optimize time. 

In [5]:
df2 = df.rename(columns={'date':'year'})
df2.head()

,Name,contract_years,salary,AAV,minor_league,catcher,year,new_team,former_team,Age,...,SB_2,BB%_2,K%_2,AVG_2,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2
0,Shohei Ohtani,10,700.0,70.000000,0,0,2024,LAD,LAA,29,...,11,0.108108,0.241742,0.273038,0.355856,0.518771,6,9,160,0.874627
1,Aaron Judge,9,360.0,40.000000,0,0,2023,NYY,NYY,31,...,6,0.118483,0.249605,0.287273,0.372828,0.543636,16,1,158,0.916464
2,Carlos Correa,3,105.3,35.100000,0,0,2022,MIN,HOU,27,...,1,0.109034,0.233645,0.278571,0.358255,0.567857,8,0,78,0.926113
3,Anthony Rendon,7,245.0,35.000000,0,0,2020,LAA,WSN,30,...,2,0.092127,0.137353,0.308129,0.373534,0.534972,5,1,163,0.908506
4,Carlos Correa,6,200.0,33.333333,0,0,2023,MIN,MIN,28,...,0,0.117188,0.181250,0.279279,0.365625,0.484685,16,0,155,0.850310


#### The Shohei Ohtani case. 

The japanese superstar signed a record 700M-10year deal with the LA Dodgers this season (2024). The contract itself has very special characteristics that make it unique. First, he is the first player since Babe Ruth to play both pitcher and batter at an elite level. The Dodgers are essentially getting two players extremely talented players in one, except for this year that he won't be able to pitch as he is recovering from a UCL injury. The record AAV for a pitcher excluding Ohtani is 43.3M for Justin Verlander and Max Scherzer. For a batter is 40M per season for Aaron Judge signed last year. 

But, Ohtani is not getting paid 70M dollars per year. The Dodgers deffered 680 million to be paid starting in 2034, when the contract expires. Ohtani is receiving 2M per season and will get 68 every year starting in 10 years from now. As explained in the paper, today's money won't be worth the same in 10 years. He is not getting 700 million dollars in 2024, he will be getting way less. 

We could solve this in two ways: Take Ohtani's accumulated WAR from the last 3 years (28.3) and divide it into two: Offensive WAR (14.3) and Pitching WAR (14.2) and calculate how much the Dodgers are paying Ohtani according to his different abilities, which will be approximately 35M for each. Right up there with what the top players at each position have received. 


The next proposed solution is to take the the annual competitive balance tax (CBT, or luxury tax) figure for the Dodgers, which is approximately 46.6M and use it as a definitive number for both pitching and batting. This estimation to what the Dodgers will be paying him taking interest rates into account. In average MLB salaries have increased 3.5% per year which would make Ohtani's 70 million, 50 million in 2024. 

For this work I will use the first solution as I see it more fitting for both the pitching models and the batting models. 

I had already converted the 28.5 WAR to 14.3 in the Data Preprocessing step. 

Now I will convert the salary and AAV to match its position valuation.



In [6]:

shohei_index = df2.index[df2['Name'] == 'Shohei Ohtani'].tolist()[0]
df2.loc[shohei_index, 'salary'] *= 0.501754386
df2.loc[shohei_index, 'AAV'] *= 0.501754386


In [7]:
df2.head()

,Name,contract_years,salary,AAV,minor_league,catcher,year,new_team,former_team,Age,...,SB_2,BB%_2,K%_2,AVG_2,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2
0,Shohei Ohtani,10,351.22807,35.122807,0,0,2024,LAD,LAA,29,...,11,0.108108,0.241742,0.273038,0.355856,0.518771,6,9,160,0.874627
1,Aaron Judge,9,360.00000,40.000000,0,0,2023,NYY,NYY,31,...,6,0.118483,0.249605,0.287273,0.372828,0.543636,16,1,158,0.916464
2,Carlos Correa,3,105.30000,35.100000,0,0,2022,MIN,HOU,27,...,1,0.109034,0.233645,0.278571,0.358255,0.567857,8,0,78,0.926113
3,Anthony Rendon,7,245.00000,35.000000,0,0,2020,LAA,WSN,30,...,2,0.092127,0.137353,0.308129,0.373534,0.534972,5,1,163,0.908506
4,Carlos Correa,6,200.00000,33.333333,0,0,2023,MIN,MIN,28,...,0,0.117188,0.181250,0.279279,0.365625,0.484685,16,0,155,0.850310


In [8]:
df2.dropna(inplace=True)

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              677 non-null    object 
 1   contract_years    677 non-null    int64  
 2   salary            677 non-null    float64
 3   AAV               677 non-null    float64
 4   minor_league      677 non-null    int64  
 5   catcher           677 non-null    int64  
 6   year              677 non-null    int64  
 7   new_team          677 non-null    object 
 8   former_team       677 non-null    object 
 9   Age               677 non-null    int64  
 10  WAR3              677 non-null    float64
 11  Yrs               677 non-null    int64  
 12  career_games      677 non-null    int64  
 13  stayed_same_team  677 non-null    int64  
 14  G                 677 non-null    int64  
 15  PA                677 non-null    int64  
 16  HR                677 non-null    int64  
 1

### Interest Rates

As mentioned before in the Ohtani case, money is not worth the same every year. This will convert every contract to its 2024 value. The coefficients have been calculated previously taking into account the entire salary mass in MLB and the change per year. 

In [10]:
df2.loc[df2['year'] == 2024, 'salary'] *= 1
df2.loc[df2['year'] == 2023, 'salary'] *= 0.994232329
df2.loc[df2['year'] == 2022, 'salary'] *= 1.097132508
df2.loc[df2['year'] == 2020, 'salary'] *= 1.187629677
df2.loc[df2['year'] == 2019, 'salary'] *= 1.188733275
df2.loc[df2['year'] == 2018, 'salary'] *= 1.183309539
df2.loc[df2['year'] == 2017, 'salary'] *= 1.171102114
df2.loc[df2['year'] == 2016, 'salary'] *= 1.231297408
df2.head()

,Name,contract_years,salary,AAV,minor_league,catcher,year,new_team,former_team,Age,...,SB_2,BB%_2,K%_2,AVG_2,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2
0,Shohei Ohtani,10,351.228070,35.122807,0,0,2024,LAD,LAA,29,...,11,0.108108,0.241742,0.273038,0.355856,0.518771,6,9,160,0.874627
1,Aaron Judge,9,357.923638,40.000000,0,0,2023,NYY,NYY,31,...,6,0.118483,0.249605,0.287273,0.372828,0.543636,16,1,158,0.916464
2,Carlos Correa,3,115.528053,35.100000,0,0,2022,MIN,HOU,27,...,1,0.109034,0.233645,0.278571,0.358255,0.567857,8,0,78,0.926113
3,Anthony Rendon,7,290.969271,35.000000,0,0,2020,LAA,WSN,30,...,2,0.092127,0.137353,0.308129,0.373534,0.534972,5,1,163,0.908506
4,Carlos Correa,6,198.846466,33.333333,0,0,2023,MIN,MIN,28,...,0,0.117188,0.181250,0.279279,0.365625,0.484685,16,0,155,0.850310


In [11]:
df2.loc[df2['year'] == 2024, 'AAV'] *= 1
df2.loc[df2['year'] == 2023, 'AAV'] *= 0.994232329
df2.loc[df2['year'] == 2022, 'AAV'] *= 1.097132508
df2.loc[df2['year'] == 2020, 'AAV'] *= 1.187629677
df2.loc[df2['year'] == 2019, 'AAV'] *= 1.188733275
df2.loc[df2['year'] == 2018, 'AAV'] *= 1.183309539
df2.loc[df2['year'] == 2017, 'AAV'] *= 1.171102114
df2.loc[df2['year'] == 2016, 'AAV'] *= 1.231297408
df2.head()

,Name,contract_years,salary,AAV,minor_league,catcher,year,new_team,former_team,Age,...,SB_2,BB%_2,K%_2,AVG_2,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2
0,Shohei Ohtani,10,351.228070,35.122807,0,0,2024,LAD,LAA,29,...,11,0.108108,0.241742,0.273038,0.355856,0.518771,6,9,160,0.874627
1,Aaron Judge,9,357.923638,39.769293,0,0,2023,NYY,NYY,31,...,6,0.118483,0.249605,0.287273,0.372828,0.543636,16,1,158,0.916464
2,Carlos Correa,3,115.528053,38.509351,0,0,2022,MIN,HOU,27,...,1,0.109034,0.233645,0.278571,0.358255,0.567857,8,0,78,0.926113
3,Anthony Rendon,7,290.969271,41.567039,0,0,2020,LAA,WSN,30,...,2,0.092127,0.137353,0.308129,0.373534,0.534972,5,1,163,0.908506
4,Carlos Correa,6,198.846466,33.141078,0,0,2023,MIN,MIN,28,...,0,0.117188,0.181250,0.279279,0.365625,0.484685,16,0,155,0.850310


In [12]:
df.count()

Name                677
contract_years      677
salary              677
AAV                 677
minor_league        677
catcher             677
date                677
new_team            677
former_team         677
Age                 677
WAR3                677
Yrs                 677
career_games        677
stayed_same_team    677
G                   677
PA                  677
HR                  677
R                   677
RBI                 677
SB                  677
BB%                 677
K%                  677
AVG                 677
OBP                 677
SLG                 677
GDP                 677
CS                  677
H                   677
OPS                 677
PlayerId            677
MLBAMID             677
G_2                 677
PA_2                677
HR_2                677
R_2                 677
RBI_2               677
SB_2                677
BB%_2               677
K%_2                677
AVG_2               677
OBP_2               677
SLG_2           

#### Drop minor league players

Minor league contracts have always the same AAV with some rare exceptions. Having these contracts won't help us with trying to predict a value we know already. The objective is to predict which players are worth major league contracts. 

In [13]:
677-(df2['minor_league'] == 1).sum()

421

In [14]:
df3 = df2[df2['minor_league'] != 1]
df3.head()

,Name,contract_years,salary,AAV,minor_league,catcher,year,new_team,former_team,Age,...,SB_2,BB%_2,K%_2,AVG_2,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2
0,Shohei Ohtani,10,351.228070,35.122807,0,0,2024,LAD,LAA,29,...,11,0.108108,0.241742,0.273038,0.355856,0.518771,6,9,160,0.874627
1,Aaron Judge,9,357.923638,39.769293,0,0,2023,NYY,NYY,31,...,6,0.118483,0.249605,0.287273,0.372828,0.543636,16,1,158,0.916464
2,Carlos Correa,3,115.528053,38.509351,0,0,2022,MIN,HOU,27,...,1,0.109034,0.233645,0.278571,0.358255,0.567857,8,0,78,0.926113
3,Anthony Rendon,7,290.969271,41.567039,0,0,2020,LAA,WSN,30,...,2,0.092127,0.137353,0.308129,0.373534,0.534972,5,1,163,0.908506
4,Carlos Correa,6,198.846466,33.141078,0,0,2023,MIN,MIN,28,...,0,0.117188,0.181250,0.279279,0.365625,0.484685,16,0,155,0.850310


In [15]:
(df3['minor_league'] == 1).sum()

0

## Feature Selection

Our goal is to predict AAV and nothing else. Not contract years or the final accumulated salary. 

Drop salary and contract years as we only need AAV. 

MLBAMID is repetitive when we have PlayerId.
New_team, former_team have already been used to know if the player stayed in the same team after signing.

Year wont be need anymore as we have converted all the values to the actual one.

WAR3 will be used until the Advanced Statistics part, 

In [16]:
columns_to_drop = ['salary', 'contract_years', 'MLBAMID', 'year', 'new_team', 'former_team', 'minor_league', 'WAR3']

df4 = df3.drop(columns=columns_to_drop)

df4.head()

,Name,AAV,catcher,Age,Yrs,career_games,stayed_same_team,G,PA,HR,...,SB_2,BB%_2,K%_2,AVG_2,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2
0,Shohei Ohtani,35.122807,0,29,6,701,0,135,599,44,...,11,0.108108,0.241742,0.273038,0.355856,0.518771,6,9,160,0.874627
1,Aaron Judge,39.769293,0,31,7,729,1,157,696,62,...,6,0.118483,0.249605,0.287273,0.372828,0.543636,16,1,158,0.916464
2,Carlos Correa,38.509351,0,27,7,752,0,148,640,26,...,1,0.109034,0.233645,0.278571,0.358255,0.567857,8,0,78,0.926113
3,Anthony Rendon,41.567039,0,30,7,916,0,146,646,34,...,2,0.092127,0.137353,0.308129,0.373534,0.534972,5,1,163,0.908506
4,Carlos Correa,33.141078,0,28,8,888,1,136,590,22,...,0,0.117188,0.181250,0.279279,0.365625,0.484685,16,0,155,0.850310


In [17]:
df4.select_dtypes(include=['number']).corr().style.background_gradient("coolwarm", vmin=-1, vmax=1)

,AAV,catcher,Age,Yrs,career_games,stayed_same_team,G,PA,HR,R,RBI,SB,BB%,K%,AVG,OBP,SLG,GDP,CS,H,OPS,PlayerId,G_2,PA_2,HR_2,R_2,RBI_2,SB_2,BB%_2,K%_2,AVG_2,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2
AAV,1.000000,-0.139391,-0.211313,-0.076589,0.114496,0.095985,0.424596,0.565345,0.652130,0.677832,0.645278,0.267949,0.220959,-0.126657,0.351846,0.417082,0.494360,0.386619,0.182230,0.603640,0.499763,0.144178,0.331838,0.445510,0.524377,0.546855,0.535005,0.103243,0.147607,-0.056067,0.290927,0.334566,0.440204,0.272691,0.154437,0.465255,0.438838
catcher,-0.139391,1.000000,0.110062,0.035002,-0.183037,-0.078322,-0.279696,-0.250320,-0.200088,-0.306734,-0.225449,-0.253194,-0.010968,0.133963,-0.146415,-0.119032,-0.144587,-0.085757,-0.230309,-0.263702,-0.145147,-0.085383,-0.261471,-0.257686,-0.166289,-0.317738,-0.208820,-0.255518,0.032531,0.166410,-0.194681,-0.133978,-0.144384,-0.106987,-0.248052,-0.283400,-0.152021
Age,-0.211313,0.110062,1.000000,0.795590,0.625565,0.082892,-0.049737,-0.070625,-0.080847,-0.092551,-0.053704,-0.100599,0.051846,-0.105525,-0.004544,0.046947,-0.012309,-0.020965,-0.130331,-0.082347,0.007301,-0.481076,-0.113914,-0.109055,-0.092501,-0.101221,-0.074718,-0.120635,0.132932,-0.096731,-0.032333,0.075746,-0.032861,-0.019811,-0.165843,-0.118759,0.000803
Yrs,-0.076589,0.035002,0.795590,1.000000,0.857663,0.154825,0.056022,0.057382,0.049529,0.041430,0.078493,-0.038686,0.051117,-0.126083,0.076434,0.097901,0.086868,0.087507,-0.054172,0.053946,0.096297,-0.498330,-0.009608,0.019577,0.022195,0.019482,0.049811,-0.086831,0.117666,-0.126073,0.001271,0.075662,0.031729,0.087350,-0.124063,0.002613,0.048762
career_games,0.114496,-0.183037,0.625565,0.857663,1.000000,0.163865,0.312448,0.321285,0.241616,0.293933,0.308740,0.079764,0.052701,-0.226056,0.185452,0.184086,0.209765,0.255813,0.096169,0.312808,0.214484,-0.413922,0.240729,0.290880,0.200770,0.279685,0.269709,0.032145,0.121955,-0.212311,0.121313,0.169519,0.154224,0.243069,0.004167,0.262199,0.171148
stayed_same_team,0.095985,-0.078322,0.082892,0.154825,0.163865,1.000000,0.052823,0.072403,0.084369,0.106200,0.096231,0.043257,-0.030927,-0.137524,0.194016,0.155254,0.181468,0.039184,0.062064,0.105597,0.184137,-0.138218,-0.022607,-0.004973,-0.035454,0.009783,-0.010859,-0.035765,-0.076655,-0.095001,0.013968,-0.031401,-0.010708,-0.027174,-0.045367,0.006608,-0.018433
G,0.424596,-0.279696,-0.049737,0.056022,0.312448,0.052823,1.000000,0.926879,0.641955,0.843531,0.790419,0.333669,0.044979,-0.193227,0.412558,0.381325,0.425767,0.629691,0.354594,0.877187,0.438053,0.015390,0.402164,0.476629,0.342094,0.473192,0.421518,0.203309,0.025834,-0.118815,0.240231,0.222212,0.263427,0.338577,0.213903,0.476976,0.269929
PA,0.565345,-0.250320,-0.070625,0.057382,0.321285,0.072403,0.926879,1.000000,0.726543,0.930160,0.877219,0.358663,0.080456,-0.224688,0.443831,0.420954,0.475936,0.694674,0.358131,0.963311,0.487832,0.045494,0.456033,0.575944,0.451992,0.583171,0.541719,0.200887,0.080604,-0.132944,0.295821,0.299623,0.342605,0.428234,0.225673,0.581068,0.354612
HR,0.652130,-0.200088,-0.080847,0.049529,0.241616,0.084369,0.641955,0.726543,1.000000,0.794993,0.896285,0.105074,0.234815,0.073860,0.325004,0.400834,0.716130,0.509760,0.074984,0.689432,0.653995,0.010289,0.320171,0.403806,0.585758,0.443538,0.538034,-0.057616,0.165902,0.120794,0.158585,0.220280,0.445759,0.276404,0.011777,0.380308,0.404887
R,0.677832,-0.306734,-0.092551,0.041430,0.293933,0.106200,0.843531,0.930160,0.794993,1.000000,0.879341,0.430234,0.160580,-0.192573,0.479528,0.498283,0.583060,0.594742,0.376505,0.920313,0.591628,0.052634,0.409986,0.524259,0.449478,0.578860,0.505597,0.231529,0.115591,-0.095514,0.288080,0.312950,0.356900,0.335195,0.257029,0.533532,0.369679


We can see K% in both years doesn't have a significant correlation with AAV. 

The same goes for BB%. BB%'s correlation is higher, but it is very highly correlated with OBP, and that makes sense as both are calculation for essentialy the same thing. OBP has a much higher correlation with AAV, so dropping BB% for both years wont impact our model. 

Our created variable 'Catcher' seems to have a very small correlation to AAV, but higher with some others. I chose to keep it.

I can't say the same thing for 'stayed_same_team'. Players seem to have accepted a very slight paycut when staying in the same team. This can be attributed, as pointed out by Libsch (2018) to players familiarity with the city, and an already established position. Basically they gave up very little money for comfort. I chose to keep it in the mean time. 

CS AND SB will be kept for now. 

Finally, from Yrs / career_games / Age, the one that shows a bigger correlation with AAV is age. It doesn't make sense to keep all three, just AGE. 

In [18]:
columns_to_drop2 = ['BB%', 'BB%_2', 'Yrs', 'career_games', 'K%', 'K%_2']

df5 = df4.drop(columns=columns_to_drop2)

df5.head()

,Name,AAV,catcher,Age,stayed_same_team,G,PA,HR,R,RBI,...,R_2,RBI_2,SB_2,AVG_2,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2
0,Shohei Ohtani,35.122807,0,29,0,135,599,44,102,95,...,90,95,11,0.273038,0.355856,0.518771,6,9,160,0.874627
1,Aaron Judge,39.769293,0,31,1,157,696,62,133,131,...,89,98,6,0.287273,0.372828,0.543636,16,1,158,0.916464
2,Carlos Correa,38.509351,0,27,0,148,640,26,104,92,...,42,59,1,0.278571,0.358255,0.567857,8,0,78,0.926113
3,Anthony Rendon,41.567039,0,30,0,146,646,34,117,126,...,88,92,2,0.308129,0.373534,0.534972,5,1,163,0.908506
4,Carlos Correa,33.141078,0,28,1,136,590,22,70,64,...,104,92,0,0.279279,0.365625,0.484685,16,0,155,0.850310


In [33]:
df5.to_csv('df5.csv', index=False)

I will select all of the features in our updated df except the target and PlayerId

In [19]:
target = "AAV"
features = [col for col in df5.columns if col != target and col != "PlayerId" and col!="Name"]
X, y = df5[features], df5[target]

## Train-Test split

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (336, 29)
X_test shape: (85, 29)
y_train shape: (336,)
y_test shape: (85,)


### Linear Model

In [21]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression()

In [22]:
linear_model.coef_, linear_model.intercept_

(array([ 2.69546352e+00, -3.62579121e-01,  4.90732858e-01, -3.01334451e-02,
        -5.76155605e-02,  2.34224771e-01,  1.97143129e-01,  3.88000302e-02,
         1.04367895e-01, -3.54457488e+01, -2.11314618e+03, -2.17338233e+03,
         2.19646730e-03, -2.21473521e-01,  1.68387517e-01,  2.15840024e+03,
        -4.63558603e-02,  9.15763706e-03,  1.90909060e-02,  8.01329511e-02,
         4.00999055e-02, -1.21259909e-02, -5.35878404e+00, -5.99980855e+02,
        -6.09411267e+02, -7.53550957e-02, -1.59578995e-03, -2.35257662e-02,
         6.11265625e+02]),
 10.871155928546575)

In [23]:
linear_y_pred = linear_model.predict(X_test)

results_df = X_test.copy()
results_df["y_real"] = y_test
results_df["y_pred"] = linear_y_pred.astype(int)
results_df["err"] = results_df["y_real"] - results_df["y_pred"]
results_df["%_err"] = results_df["err"] / results_df["y_real"] * 100
results_df

,catcher,Age,stayed_same_team,G,PA,HR,R,RBI,SB,AVG,...,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2,y_real,y_pred,err,%_err
433,0,31,1,84,221,9,34,33,3,0.241379,...,0.278997,0.344595,7,0,73,0.623591,1.187630,4,-2.812370,-236.805326
165,0,36,0,134,495,12,74,48,43,0.248899,...,0.306233,0.439883,5,8,88,0.746116,7.026613,10,-2.973387,-42.316084
17,0,31,0,160,678,26,70,93,7,0.257600,...,0.315888,0.490982,18,3,133,0.806870,23.500000,11,12.500000,53.191489
179,0,32,0,117,436,6,43,39,2,0.251269,...,0.326165,0.406375,16,4,128,0.732539,6.240850,2,4.240850,67.953082
4,0,28,1,136,590,22,70,64,0,0.291188,...,0.365625,0.484685,16,0,155,0.850310,33.141078,19,14.141078,42.669336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,0,35,0,92,353,21,45,58,4,0.246875,...,0.276190,0.400697,2,2,61,0.676887,3.000000,6,-3.000000,-100.000000
90,0,31,1,118,453,13,45,53,18,0.231144,...,0.314346,0.409195,5,9,111,0.723541,13.467674,6,7.467674,55.448878
341,0,35,1,52,190,6,20,27,0,0.257310,...,0.337461,0.486111,10,1,76,0.823572,2.375259,3,-0.624741,-26.301997
81,0,31,1,159,667,47,94,108,2,0.256117,...,0.310092,0.448819,12,0,133,0.758911,14.638776,19,-4.361224,-29.792268


## Evaluation Metrics 

In [24]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

print(f"RMSE: {mean_squared_error(y_test, linear_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, linear_y_pred)}")
print(f"R^2: {r2_score(y_test, linear_y_pred)}")

RMSE: 5.047302241104886
MAPE: 1.0717581747175853
R^2: 0.6006211023803794


The RMSE is telling us the average error in the predictions: we are failing by 5M on average. The MAPE tells us about the average percentage error, since 5M is not the same in a player's AAV of 200k than for a 40M player AAV : in our case we are failing by 107% on average. Finally the R^2 score is the Pearson's coefficient of predictions against reality: 1 would be a perfect model, while 0 is the worst possible scenario. 

## XGBoost

In [25]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor()

xgb_model.fit(X_train, y_train)
xgb_y_pred = xgb_model.predict(X_test)

results_df_xgb = X_test.copy()
results_df_xgb["y_real"] = y_test
results_df_xgb["y_pred"] = xgb_y_pred.astype(int)
results_df_xgb["err"] = results_df_xgb["y_real"] - results_df_xgb["y_pred"]
results_df_xgb["%_err"] = results_df_xgb["err"] / results_df_xgb["y_real"] * 100


results_df_xgb


,catcher,Age,stayed_same_team,G,PA,HR,R,RBI,SB,AVG,...,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2,y_real,y_pred,err,%_err
433,0,31,1,84,221,9,34,33,3,0.241379,...,0.278997,0.344595,7,0,73,0.623591,1.187630,3,-1.812370,-152.603994
165,0,36,0,134,495,12,74,48,43,0.248899,...,0.306233,0.439883,5,8,88,0.746116,7.026613,5,2.026613,28.841958
17,0,31,0,160,678,26,70,93,7,0.257600,...,0.315888,0.490982,18,3,133,0.806870,23.500000,9,14.500000,61.702128
179,0,32,0,117,436,6,43,39,2,0.251269,...,0.326165,0.406375,16,4,128,0.732539,6.240850,2,4.240850,67.953082
4,0,28,1,136,590,22,70,64,0,0.291188,...,0.365625,0.484685,16,0,155,0.850310,33.141078,14,19.141078,57.756353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,0,35,0,92,353,21,45,58,4,0.246875,...,0.276190,0.400697,2,2,61,0.676887,3.000000,6,-3.000000,-100.000000
90,0,31,1,118,453,13,45,53,18,0.231144,...,0.314346,0.409195,5,9,111,0.723541,13.467674,6,7.467674,55.448878
341,0,35,1,52,190,6,20,27,0,0.257310,...,0.337461,0.486111,10,1,76,0.823572,2.375259,3,-0.624741,-26.301997
81,0,31,1,159,667,47,94,108,2,0.256117,...,0.310092,0.448819,12,0,133,0.758911,14.638776,19,-4.361224,-29.792268


In [26]:
print(f"RMSE: {mean_squared_error(y_test, xgb_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, xgb_y_pred)}")
print(f"R^2: {r2_score(y_test, xgb_y_pred)}")

RMSE: 6.070107870403328
MAPE: 0.9180397311648749
R^2: 0.4223572750548099


## Ridge Regression

In [27]:
from sklearn.linear_model import Ridge

ridge_model = Ridge()

ridge_model.fit(X_train, y_train)

ridge_y_pred = ridge_model.predict(X_test)

results_df_ridge = X_test.copy()
results_df_ridge["y_real"] = y_test
results_df_ridge["y_pred"] = ridge_y_pred.astype(int)
results_df_ridge["err"] = results_df_ridge["y_real"] - results_df_ridge["y_pred"]
results_df_ridge["%_err"] = results_df_ridge["err"] / results_df_ridge["y_real"] * 100

results_df_ridge

,catcher,Age,stayed_same_team,G,PA,HR,R,RBI,SB,AVG,...,OBP_2,SLG_2,GDP_2,CS_2,H_2,OPS_2,y_real,y_pred,err,%_err
433,0,31,1,84,221,9,34,33,3,0.241379,...,0.278997,0.344595,7,0,73,0.623591,1.187630,5,-3.812370,-321.006657
165,0,36,0,134,495,12,74,48,43,0.248899,...,0.306233,0.439883,5,8,88,0.746116,7.026613,11,-3.973387,-56.547692
17,0,31,0,160,678,26,70,93,7,0.257600,...,0.315888,0.490982,18,3,133,0.806870,23.500000,11,12.500000,53.191489
179,0,32,0,117,436,6,43,39,2,0.251269,...,0.326165,0.406375,16,4,128,0.732539,6.240850,2,4.240850,67.953082
4,0,28,1,136,590,22,70,64,0,0.291188,...,0.365625,0.484685,16,0,155,0.850310,33.141078,18,15.141078,45.686739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,0,35,0,92,353,21,45,58,4,0.246875,...,0.276190,0.400697,2,2,61,0.676887,3.000000,8,-5.000000,-166.666667
90,0,31,1,118,453,13,45,53,18,0.231144,...,0.314346,0.409195,5,9,111,0.723541,13.467674,6,7.467674,55.448878
341,0,35,1,52,190,6,20,27,0,0.257310,...,0.337461,0.486111,10,1,76,0.823572,2.375259,4,-1.624741,-68.402663
81,0,31,1,159,667,47,94,108,2,0.256117,...,0.310092,0.448819,12,0,133,0.758911,14.638776,19,-4.361224,-29.792268


In [28]:
print(f"RMSE: {mean_squared_error(y_test, ridge_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, ridge_y_pred)}")
print(f"R^2: {r2_score(y_test, ridge_y_pred)}")

RMSE: 5.206564434750842
MAPE: 1.0008860913032451
R^2: 0.5750195175881649


In [32]:
import statsmodels.api as sm

# Add a constant column to X_train for intercept term
X_train_with_const = sm.add_constant(X_train)

# Initialize and fit Ridge Regression model using statsmodels
ridge_model_stats = sm.OLS(y_train, X_train_with_const)
ridge_results = ridge_model_stats.fit_regularized(alpha=1)  # Set alpha for Ridge regularization

# Print the coefficients
print("Coefficients:")
print(ridge_results.params)

# Print the standard errors of the coefficients
print("\nStandard Errors:")
print(ridge_results.bse)

# Print the p-values of the coefficients
print("\nP-values:")
print(ridge_results.pvalues)

# Print the R-squared value
print("\nR-squared:")
print(ridge_results.rsquared)

# Perform ANOVA
anova_table = sm.stats.anova_lm(ridge_results)
print("\nANOVA table:")
print(anova_table)


Coefficients:
const               0.000000
catcher             0.000000
Age                 0.000000
stayed_same_team    0.000000
G                  -0.075268
PA                  0.009069
HR                  0.285680
R                   0.152693
RBI                -0.020603
SB                  0.022886
AVG                 0.000000
OBP                 0.000000
SLG                 0.000000
GDP                -0.008179
CS                  0.000000
H                  -0.006714
OPS                 0.000000
G_2                -0.028491
PA_2                0.002753
HR_2                0.024013
R_2                 0.087362
RBI_2               0.000000
SB_2                0.000000
AVG_2               0.000000
OBP_2               0.000000
SLG_2               0.000000
GDP_2               0.000000
CS_2                0.000000
H_2                -0.001867
OPS_2               0.000000
dtype: float64

Standard Errors:


AttributeError: 'RegularizedResults' object has no attribute 'bse'

# Extracting predictors (X) and target variable (y) from df3
X = df3.drop(columns=['AAV'])  # Assuming 'target_column' is the name of your target variable
y = df3['AAV']

# Define the number of folds
k = 10

# Initialize a k-fold cross-validation splitter
kf = KFold(n_splits=k, shuffle=True)

# Initialize variables to store the best subset and its corresponding error
best_subset = None
best_error = float('inf')

# Loop through each possible subset of predictors
for r in range(1, len(X.columns) + 1):
    for subset in combinations(X.columns, r):
        cv_errors = []
        for train_index, val_index in kf.split(X):
            X_train, X_val = X.iloc[train_index][list(subset)], X.iloc[val_index][list(subset)]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]

            # Fit model
            model = LinearRegression()
            model.fit(X_train, y_train)

            # Predict on validation set
            y_pred = model.predict(X_val)

            # Calculate Mean Squared Prediction Error
            mse = mean_squared_error(y_val, y_pred)
            cv_errors.append(mse)

        # Average the errors across all folds to get CV error for this subset
        CV_error = np.mean(cv_errors)

        # Update best subset if CV error is lower
        if CV_error < best_error:
            best_error = CV_error
            best_subset = subset

# best_subset now contains the column names of the best subset of predictors


# Plot MSPR results against the number of variables
plt.plot(predictors, mspr_values, marker='o', linestyle='-')
plt.xlabel('Number of Variables')
plt.ylabel('Mean Squared Prediction Error (MSPR)')
plt.title('MSPR vs. Number of Variables')
plt.grid(True)
plt.show()
